<a href="https://colab.research.google.com/github/Sergio-ddf/emit-llm/blob/main/emit_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EMit Emotion Detection Task

# 1. Setup e import


In [1]:
# API Key WandB
from google.colab import userdata
import os, wandb

os.environ["WANDB_API_KEY"] = userdata.get('WANDB_KEY')
wandb.login()

wandb: Currently logged in as: ddfsergio9 (ddfsergio9-sapienza-universit-di-roma) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [2]:
# installazione pacchetti
!pip install -q -U "transformers>=4.39.0" datasets scikit-multilearn emoji wandb
!pip install -q iterative-stratification

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 590.6/590.6 kB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.4/21.4 MB 91.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 20.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.


In [3]:
# import e seed
import os, re, random, numpy as np, pandas as pd, emoji, torch, torch.nn as nn
import torch.nn as nn

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report
from sklearn.multiclass import OneVsRestClassifier

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

from datasets import Dataset, DatasetDict, Sequence, Value
from transformers import (AutoTokenizer, AutoModelForSequenceClassification,
                          TrainingArguments, Trainer)

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True; torch.backends.cudnn.benchmark = False


print(f"PyTorch    : {torch.__version__} (CUDA disponibile: {torch.cuda.is_available()})")

PyTorch    : 2.6.0+cu124 (CUDA disponibile: True)


# 2. Configurazione e percorsi


In [4]:
DATA_DIR     = ""  # path della cartella con i dati
MODEL_NAME   = "Musixmatch/umberto-commoncrawl-cased-v1"
DEVICE       = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LABELS       = ['Anger','Anticipation','Disgust','Fear','Joy',
                'Love','Neutral','Sadness','Surprise','Trust']
NUM_LABELS   = len(LABELS)


# 3. Caricamento dati e statistiche


In [5]:
train_df = pd.read_csv(os.path.join(DATA_DIR, "emit_train_A.csv"))
test_df = pd.read_csv(os.path.join(DATA_DIR, "emit_test.csv"))

print("Train A:", train_df.shape, "Test in-domain:", test_df.shape)
display(train_df.head())

# distribuzione etichette
counts = train_df[LABELS].sum().sort_values(ascending=False)
display(counts.to_frame("etichette"))


Train A: (5966, 12) Test in-domain: (1000, 14)


,id,text,Anger,Anticipation,Disgust,Fear,Joy,Love,Neutral,Sadness,Surprise,Trust
0,04b19fcefed25f6919fbbb8fe6ac4f1e,RT @user: Serve aggiungere altro? #Ulisse INIZ...,0,1,0,0,1,1,0,0,0,0
1,6320e808285b5e1613dbd28580affd21,RT @user: La differenza tra chi ha studiato ed...,0,0,0,0,0,0,1,0,0,0
2,abdddf2c6231ca527bbaf808e8c11085,Metà della sobrietà di #IlariaCucchi dovrebbe ...,0,0,0,0,0,0,0,0,0,1
3,8b6a76b34ed6bdb23d61ba9ef386ecb8,LENÙ PER PIACERE SCAPPA STO IN ANSIA PER TE TI...,0,1,0,1,0,0,0,0,0,0
4,913743b451b8178fa138c7b27f26fac6,@user Nooooo!!! La signora Cecchini 😢 #DonMatt...,0,0,0,0,0,0,0,1,0,0


,etichette
Trust,1665
Neutral,1322
Disgust,874
Joy,650
Love,633
Surprise,591
Anticipation,547
Sadness,545
Anger,367
Fear,91


# 4. Pulizia testo


In [6]:
URL, USER, TAG = "<URL>", "<USER>", "<HASHTAG>"

def clean(text: str) -> str:
    """Minimal text-normalizer per social media italiani."""
    t = re.sub(r'https?://\S+', URL, text)          # link → <URL>
    t = re.sub(r'@\w+', USER, t)                    # mention → <USER>
    t = re.sub(r'#(\w+)', TAG + r' \1', t)          # hashtag → <HASHTAG> parola
    t = emoji.demojize(t, language='it')            # 😀 → :grinning_face:
    return t.strip()

train_df["text_clean"] = train_df["text"].astype(str).map(clean)
test_df["text_clean"]  = test_df["text"].astype(str).map(clean)


# 5. Split stratificato 90/10


In [7]:
# split multilabel
X = train_df["text_clean"].values
Y = train_df[LABELS].values

msss = MultilabelStratifiedShuffleSplit(test_size=0.1, random_state=SEED)
train_idx, val_idx = next(msss.split(X, Y))

train_split = train_df.iloc[train_idx].reset_index(drop=True)
val_split   = train_df.iloc[val_idx].reset_index(drop=True)

print(f"→ Train {train_split.shape}  Valid {val_split.shape}")


→ Train (5381, 13)  Valid (585, 13)


# 6. Baseline TF-IDF + Logistic Regression


In [8]:
vec = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
X_tr = vec.fit_transform(train_split['text_clean'])
X_va = vec.transform(val_split['text_clean'])

y_tr = train_split[LABELS].values
y_va = val_split[LABELS].values

clf = OneVsRestClassifier(LogisticRegression(max_iter=1000, random_state=42))
clf.fit(X_tr, y_tr)
y_pr = clf.predict(X_va)

print("TF-IDF+LR macro-F1:", f1_score(y_va, y_pr, average='macro'))
print(classification_report(y_va, y_pr, target_names=LABELS, zero_division=0))


TF-IDF+LR macro-F1: 0.1598687722003072
              precision    recall  f1-score   support

       Anger       0.00      0.00      0.00        37
Anticipation       0.71      0.09      0.16        55
     Disgust       0.00      0.00      0.00        87
        Fear       0.00      0.00      0.00         9
         Joy       0.80      0.12      0.21        65
        Love       0.72      0.21      0.32        63
     Neutral       0.69      0.14      0.23       132
     Sadness       1.00      0.09      0.17        55
    Surprise       0.80      0.07      0.12        59
       Trust       0.81      0.25      0.38       167

   micro avg       0.77      0.13      0.22       729
   macro avg       0.55      0.10      0.16       729
weighted avg       0.64      0.13      0.21       729
 samples avg       0.16      0.14      0.14       729



# 7. Preparazione HF Dataset


In [9]:
model_ckpt = "Musixmatch/umberto-commoncrawl-cased-v1"
tokenizer  = AutoTokenizer.from_pretrained(model_ckpt, use_fast=True)

for df in (train_split, val_split, test_df):
    df["labels"] = df[LABELS].apply(lambda row: [float(x) for x in row], axis=1)

def tokenize(batch):
    tokens = tokenizer(
        batch["text_clean"],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    tokens["labels"] = batch["labels"]
    return tokens

def to_ds(df):
    ds = Dataset.from_pandas(df[["text_clean", "labels"]])
    ds = ds.cast_column("labels", Sequence(feature=Value("float32")))
    return ds.map(tokenize, batched=True)

hf_train = to_ds(train_split)
hf_val   = to_ds(val_split)
hf_test  = to_ds(test_df)

data = DatasetDict({"train": hf_train, "validation": hf_val, "test": hf_test})
data = data.remove_columns(["text_clean"])


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/508 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/794k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.68M [00:00<?, ?B/s]

Casting the dataset:   0%|          | 0/5381 [00:00<?, ? examples/s]

Map:   0%|          | 0/5381 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/585 [00:00<?, ? examples/s]

Map:   0%|          | 0/585 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

# 8. Fine-tuning UmBERTo con BCE pesata


In [10]:
# Caricamento del modello base
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt,
    num_labels=NUM_LABELS,
    problem_type="multi_label_classification"
)

# calcolo del bias iniziale
label_sums = train_split[LABELS].sum()
pi = label_sums / len(train_split)
pi_tensor = torch.tensor(pi.values, dtype=torch.float32)
bias_init = torch.log(pi_tensor / (1.0 - pi_tensor + 1e-8))

# inizializzazione del bias del layer finale
model.classifier.out_proj.bias.data = bias_init.to(model.device)

# Calcolo pos_weight per classi sbilanciate
neg_counts = len(train_split) - label_sums
pos_weight_np = (neg_counts / label_sums).values
pos_weight_tensor = torch.tensor(pos_weight_np, dtype=torch.float32, device=model.device)


pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of CamembertForSequenceClassification were not initialized from the model checkpoint at Musixmatch/umberto-commoncrawl-cased-v1 and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# iperparametri + Warmup + Scheduler

args = TrainingArguments(
    output_dir               = "./ubert-emotions",
    eval_strategy            = "epoch",
    save_strategy            = "epoch",
    logging_strategy         = "steps",
    logging_steps            = 100,
    learning_rate            = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size  = 32,
    num_train_epochs         = 5,
    weight_decay             = 0.01,
    warmup_ratio             = 0.1,
    lr_scheduler_type        = "cosine",
    metric_for_best_model    = "eval_f1",
    load_best_model_at_end   = True,
    seed                     = SEED,
    report_to                = ["wandb"],
)


In [12]:
# metrica + CustomTrainer

def compute_metrics(pred):
    logits, labels = pred
    probs  = torch.sigmoid(torch.tensor(logits))
    y_pred = (probs > 0.5).int().cpu().numpy()
    y_true = labels
    return {"f1": f1_score(y_true, y_pred, average="macro")}

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = inputs.pop("labels").float().to(model.device)
        # Forward pass
        outputs = model(**inputs)
        logits = outputs.logits
        pw = pos_weight_tensor.to(logits.device)
        # BCEWithLogits con pos_weight
        loss_fct = nn.BCEWithLogitsLoss(pos_weight=pw)
        loss = loss_fct(logits, labels)
        return (loss, outputs) if return_outputs else loss

trainer = CustomTrainer(
    model           = model,
    args            = args,
    train_dataset   = data["train"],
    eval_dataset    = data["validation"],
    tokenizer       = tokenizer,
    compute_metrics = compute_metrics,
)

trainer.train()


<ipython-input-12-78b9c0babe49>:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `CustomTrainer.__init__`. Use `processing_class` instead.
  trainer = CustomTrainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,F1
1,1.405300,1.250134,0.277138
2,1.015000,0.997040,0.394441
3,0.803500,0.953977,0.468275
4,0.688000,0.927342,0.454342
5,0.623500,0.929921,0.470032


TrainOutput(global_step=1685, training_loss=0.988110460230434, metrics={'train_runtime': 666.109, 'train_samples_per_second': 40.391, 'train_steps_per_second': 2.53, 'total_flos': 1769877855137280.0, 'train_loss': 0.988110460230434, 'epoch': 5.0})

In [13]:
# Calibrazione delle soglie su validation

# 1) probabilità predette sulla validation
val_logits = trainer.predict(data["validation"]).predictions
val_probs  = torch.sigmoid(torch.tensor(val_logits)).cpu().numpy()

y_val_true = val_split[LABELS].values

# ricerca soglia ottimale per ciascuna classe
best_thresholds = {}
for i, label in enumerate(LABELS):
    best_f1, best_thr = 0.0, 0.5
    for thr in np.arange(0.1, 0.9, 0.01):
        y_pred_i = (val_probs[:, i] > thr).astype(int)
        f1 = f1_score(y_val_true[:, i], y_pred_i)
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    best_thresholds[label] = best_thr

print("Soglie ottimali per classe:")
for lbl, thr in best_thresholds.items():
    print(f"  {lbl:12s}: {thr:.2f}")


Soglie ottimali per classe:
  Anger       : 0.77
  Anticipation: 0.60
  Disgust     : 0.57
  Fear        : 0.68
  Joy         : 0.68
  Love        : 0.69
  Neutral     : 0.54
  Sadness     : 0.69
  Surprise    : 0.66
  Trust       : 0.44


In [14]:
# inference sul test con soglie calibrate

# Probabilità sul test set
test_logits = trainer.predict(data["test"]).predictions
test_probs  = torch.sigmoid(torch.tensor(test_logits)).cpu().numpy()

threshold_list = np.array([best_thresholds[l] for l in LABELS])
binary_preds   = (test_probs > threshold_list).astype(int)

submission = pd.DataFrame({
    "id": test_df["id"],
    "labels": [
        " ".join([lbl for lbl, flag in zip(LABELS, row) if flag])
        if row.sum() > 0 else "Neutral"
        for row in binary_preds
    ]
})

submission.to_csv("emit_submission_calibrated.csv", index=False)
print("CSV salvato in emit_submission_calibrated.csv")


CSV salvato in emit_submission_calibrated.csv


In [15]:
# valutazione del modello sul CSV di submission

# caricamento file .csv
SUBM_PATH = "emit_submission_calibrated.csv"
TRUE_TEST = "emit_test.csv"

subm_df = pd.read_csv(SUBM_PATH)
true_df = pd.read_csv(TRUE_TEST)

y_true = true_df[LABELS].values.astype(int)

def parse_labels(label_str):
    if label_str.strip() == "Neutral":
        # vettore tutto zero tranne Neutral=1
        vec = np.zeros(len(LABELS), dtype=int)
        vec[LABELS.index("Neutral")] = 1
        return vec
    toks = label_str.split()
    vec = np.array([1 if lbl in toks else 0 for lbl in LABELS], dtype=int)
    return vec

y_pred = np.vstack(subm_df["labels"].map(parse_labels).values)

# calcolo delle metriche
print("Macro-F1 generale:", f1_score(y_true, y_pred, average="macro"))
print("\nClassification Report (per classe):\n")
print(classification_report(y_true, y_pred, target_names=LABELS, digits=4))


Macro-F1 generale: 0.5167939780055829

Classification Report (per classe):

              precision    recall  f1-score   support

       Anger     0.3008    0.6607    0.4134        56
Anticipation     0.5700    0.6706    0.6162        85
     Disgust     0.4403    0.7818    0.5633       165
        Fear     0.4000    0.4615    0.4286        13
         Joy     0.4387    0.6800    0.5333       100
        Love     0.3722    0.6505    0.4735       103
     Neutral     0.5784    0.7381    0.6485       210
     Sadness     0.3867    0.6105    0.4735        95
    Surprise     0.3200    0.4706    0.3810       102
       Trust     0.5381    0.7794    0.6366       272

   micro avg     0.4579    0.6969    0.5527      1201
   macro avg     0.4345    0.6504    0.5168      1201
weighted avg     0.4684    0.6969    0.5573      1201
 samples avg     0.5272    0.7032    0.5715      1201

